In [9]:
import os
import time
import pygame
from io import BytesIO
from gtts import gTTS
import speech_recognition as sr
from langchain_mistralai import ChatMistralAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import chromadb
from chromadb.utils import embedding_functions
from langchain_huggingface import HuggingFaceEmbeddings

from db_search import get_info as search

In [10]:
llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    api_key="I5uzjp4ZXUioIEM7hdYgdJtpv4NUlWov"
)

In [11]:
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("_"*100)
        try:
            audio = recognizer.listen(source, timeout=300)
            user_input = recognizer.recognize_google(audio)
            print(f"You said: {user_input}")
            return user_input
        except sr.UnknownValueError:
            return None
        except sr.WaitTimeoutError:
            return None

In [12]:
def speak_text(text):
    try:
        tts = gTTS(text=text, lang='en')
        mp3_fp = BytesIO()  # Create in-memory file object
        tts.write_to_fp(mp3_fp)
        mp3_fp.seek(0)  # Important: rewind to the beginning of the file
    
        pygame.mixer.init()  # Initialize mixer (do this ONCE in your program)
        pygame.mixer.music.load(mp3_fp)
        pygame.mixer.music.play()

        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)

    except Exception as e:
        print(f"Error: {e}")

In [13]:
def get_policies():
    embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")
    client = chromadb.PersistentClient(path="./vector_db/")
    collection = client.get_or_create_collection(
        name='policies',
        embedding_function=embedding_function
    )
        
    query = 'All policies and guidelines related to the customer'
    
    results = collection.query(
        query_texts=[query],
        n_results=5,
        include=["documents", "metadatas"]
    )

    return ''.join([rules for rules in results['documents'][0]])

In [14]:
def get_response(user_query, phone) -> str:

    policies = get_policies()
    user_data = search(phone)
    
    #Also, make sure to answer the following questions considering the history of the conversation:
    llm_query_res_template = """
        You are an intelligent virtual financial assistant, directly engaging with {first} {last} about their loan repayments queries.
        They are a {gender} having an income of INR {income} with a {credit} Credit Score. 
        They have availed a {loan_type} of amount INR {loan} with a {interest}% interest rate and loan processing fee of INR {process_fee}.
        They have planned to start repaying their loan in installments of amount INR {installment} from {start_date} over a period of {tenure} months.
        They have yet to repay a loan of INR {balance} through {payment_mode}.
        They already have {late_payment} late payment(s) and their last date of loan repayment is {last_date}. 
        
        Your role is to help manage their loan repayment and answer financial questions in a clear and precise way. 
        Communicate in a friendly, professional, authoritative manner and address the customer directly ("you") with concise responses.
        Make sure you communicate with the user in such a way that your response should always lead to their loan repayment.
        While responding to the user's queries consider their provided information to give accurate answers
        Based on the user question, you should respond in a short way. Do not write much; it should be short and precise.
        
        Answer the question while adhering to the following policies: {policies}. 
        
        Instructions:
        1. Use precise financial language and ensure clear, accurate information.
        2. If the user is willing to pay the loan then please provide this link '''https://paymentUSER1UDN.com'''. Do not send the link until user requests or user wants to pay the loan.
        3. If the customer is struggling, provide options like grace periods, payment restructuring, or deadline extensions considering their income, number of late repayment and loan amount yet to be repayed.
        4. Keep responses short and to the point.
        5. Ensure confidentiality and remind the customer to keep their payment details secure.
        6. You can only extend the last loan repayment date by a maximum of 10 days if user requests for grace periods or deadline extensions considering their income, number of late repayment and loan amount yet to be repayed.
        7. If the question cannot be answered using the information provided, reply with "Sorry, but I am unable to answer this query". 

        Question: {user_query}
        Answer:
    """

    prompt_query_res_template = ChatPromptTemplate.from_template(llm_query_res_template)
    llm_chain = prompt_query_res_template | llm | StrOutputParser()

    response = ''.join([chunk for chunk in llm_chain.stream({
        "user_query": user_query,
        "first": user_data['F_Name'][0],
        "last": user_data['L_Name'][0],
        "gender": user_data['Gender'][0],
        "income": user_data['Income'][0],
        "credit": user_data['Bureau_score'][0],
        "loan_type": user_data['Loan_type'][0],
        "loan" : user_data['Loan_amount'][0],
        "interest": user_data['Interest_Rate'][0],
        "process_fee": user_data['Loan_Processing_Fee'][0],
        "installment": user_data['Installment_Amount'][0],
        "start_date": user_data['Repayment_Start_Date'][0],
        "tenure": user_data['Repayment_tenure'][0],
        "balance": user_data['Current_balance'][0],
        "payment_mode": user_data['Repayment_mode'][0],
        "late_payment": user_data['No_of_late_payments'][0],
        "last_date": user_data['Date_of_last_payment'][0],
        "policies": policies
    })])

    return response

In [15]:
def chatbot(phone):
    print("Voice-enabled Chatbot is now running! Say 'quit' to exit.")
    while True:
        user_query = recognize_speech()
        start_time = time.time()
        if not user_query:
            continue
        if user_query.lower() == "thanks":
            print("Glad to help!")
            break
            
        response = get_response(user_query=user_query, phone=phone)
        end_time = time.time()
        print(f'Bot: {response}')
        speak_text(response)

        latency = end_time - start_time
        print(f"Response latency: {latency:.2f} seconds")

In [16]:
if __name__ == '__main__':
    chatbot(9988953565)

Voice-enabled Chatbot is now running! Say 'quit' to exit.
____________________________________________________________________________________________________
You said: can you help me with my loan payment
Bot: Of course, Myra! You've planned to start repaying your Consumer Durable Loan of INR 43431.0 with installments of INR 7468.2 from 2023-10-02 over 6 months. You also have an outstanding UPI loan of INR 23371.7. Would you like to proceed with a payment today?
Response latency: 2.61 seconds
____________________________________________________________________________________________________
You said: thanks
Glad to help!
